In [1]:
import gym
import time
import pybulletgym  # register PyBullet enviroments with open ai gym
import numpy as np
import matplotlib.pyplot as plt
from statistics import mean
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions.multivariate_normal import MultivariateNormal

np.set_printoptions(suppress=True, precision=4)

In [2]:
class Policy(nn.Module):

    def __init__(self, num_classes, lr):
        super(Policy, self).__init__()
        self.fc1 = nn.Linear(8, 64)
        self.hidden = nn.Linear(64, 64)
        self.fc2 = nn.Linear(64, num_classes)
        self.tanh = nn.Tanh()
        
        self.sigma0 = nn.Parameter(torch.tensor([0.1]))
        self.sigma1 = nn.Parameter(torch.tensor([0.1]))
        
        self.optimizer = optim.Adam(self.parameters(), lr=lr)
        
    def forward(self, x):
        
        x = self.fc1(x)
        x = self.tanh(x)
        x = self.hidden(x)
        x = self.tanh(x)
        x = self.fc2(x)
        x = self.tanh(x)
        
        sigma0 = torch.relu(self.sigma0)
        sigma1 = torch.relu(self.sigma1)
        
        if sigma0 <= 0.001: sigma0 = sigma0 + 0.001
        if sigma1 <= 0.001: sigma1 = sigma1 + 0.001

        return x, torch.diag(torch.tensor([sigma0, sigma1]))

def makePolicyNN(num_actions=2, lr=0.01):
    ''' Initialize the policy class '''
    assert isinstance(num_actions, int) and num_actions>0
    
    return Policy(num_actions, lr)

In [7]:
class PolicyGradient:
    def __init__(self, gamma, N=500, max_steps=1000):
        ''' Initialize the Reacher PyBullet environment '''
        assert isinstance(gamma, float) and 0.0<gamma<1.0, 'Invalid gamma'
        assert isinstance(N, int) and N>0
        assert isinstance(max_steps, int) and max_steps>0
        
        self.env = gym.make("modified_gym_env:ReacherPyBulletEnv-v1", rand_init=False)
        self.gamma = gamma
        self.N = N
        self.max_steps = max_steps
        self.numActions = 2
        self.numObs = 8
        self.num_steps = 0
        self.max_steps_reached = False
        
    def saveModel(self, index):
        ''' Saves the policy NN'''
        filename = 'q2_policy' + str(index) + '.pth.tar'
        torch.save(self.state, filename)
    
#     def getSigma(self, policy_network):
#         ''' Return sigma matrix of type tensor '''
#         #extract the value from the object of type parameter
#         sigma0 = policy_network.sigma0.data
#         sigma0 = torch.relu(sigma0)
        
#         sigma1 = policy_network.sigma1.data
#         sigma1 = torch.relu(sigma1)
        
#         # making sure each sigma value is not less than 1e-3
#         if sigma0 < 1e-3: sigma0 = 1e-3
#         if sigma1 < 1e-3: sigma1 = 1e-3
        
#         return torch.diag([sigma0, sigma1])
    
    def getAction(self, policy_network, state):
        ''' Return an action from a stochastic policy '''
        assert isinstance(state, np.ndarray) and len(state) == self.numObs
        
        state = torch.from_numpy(state).float()
        torque_mean, torque_sigma = policy_network(state) #forward pass
        
        # Sampling from the 2D Gaussian and calculating the actions log probability
        m = MultivariateNormal(torque_mean, torque_sigma)
        action = m.sample() #type tensor
        log_prob_of_action = m.log_prob(action) #type tensor

        return action, log_prob_of_action
    
    def runEpisode(self, policy_network):
        ''' Generate [s_t, a_t, r_t] pairs for one episode '''
        
        state = self.env.reset()
        done = False
        rewards, log_prob_of_actions = [], []
        while not(done):
            a, lpa = self.getAction(policy_network, state)
            state, r, done, info = self.env.step(np.array(a))
            r = rewards.append(r)
            log_prob_of_actions.append(lpa)
            self.num_steps += 1 #incrementing total number of steps in one iteration
            if self.num_steps >= self.max_steps: #checking
                self.max_steps_reached = True
                break;
        
        return rewards, log_prob_of_actions
    
    def doReinforcePart1(self, policy_network, verbose=False):
        ''' Improve policy by implementing vanilla version of Reinforce algo '''
        
        gamma = self.gamma
        arr_objective = []
        avg_returns = np.zeros((self.N,))
        num_episodes = np.zeros((self.N,))
        sigma = np.zeros((self.N, 2))
        
        for i in range(self.N): #improving policy for "self.N" number of iterations
            n, objective = 0, 0
            
            # Resetting variables
            self.num_steps = 0
            self.max_steps_reached = False
            
            # Looping until a maximum number of steps are taken
            while not(self.max_steps_reached):
                # Run an episode with policy and count number of steps taken
                r, log_prob_of_actions = self.runEpisode(policy_network)
                n += 1 #increasing the episode count by 1
                
                # Calculate discounted return (G_tau) and summation of log probs of actions from the episode
                G_tau = sum([ r[t] * gamma**t for t in range(len(r))])
                sum_lpa = sum(log_prob_of_actions)
                avg_returns[i] += sum(r)
                
                # Calculate objective
                objective += G_tau * sum_lpa
                assert isinstance(objective, torch.Tensor)
            
            avg_returns[i] /= n
            num_episodes[i] = n
            sigma0, sigma1 = policy_network.sigma0.data.item(), policy_network.sigma1.data.item()
            sigma[i,:] = np.array([sigma0, sigma1])
            
            # Updating policy
            policy_network.optimizer.zero_grad()
            objective = -objective/n #averaging over n episodes and flipping sign so it does gradient ascent
            objective.backward()
            policy_network.optimizer.step()
            arr_objective.append(objective.item())

            if verbose and (i%5 == 0):
                print('Iteration: {0} \t Objective: {1:.3f} \t Average reward: {2:.3f} \t Num_episodes: {3}'\
                      .format(i, objective, avg_returns[i], num_episodes[i]))
                
        self.state = { 'state_dict': policy_network.state_dict(),
                      'optimizer': policy_network.optimizer.state_dict() }

        return policy_network, arr_objective, avg_returns, num_episodes, sigma

In [ ]:
torch.manual_seed(0)
render_env = True
plot = True

policy_network = makePolicyNN(num_actions=2, lr=1e-3)
pg = PolicyGradient(gamma=0.9, N = 100, max_steps=3000)
policy_network, arr_objective, avg_returns, num_episodes, sigma = pg.doReinforcePart1(policy_network, verbose=True)

options= 
Iteration: 0 	 Objective: -159.864 	 Average reward: -26.967 	 Num_episodes: 21.0
Iteration: 5 	 Objective: -164.057 	 Average reward: -27.251 	 Num_episodes: 21.0
Iteration: 10 	 Objective: -150.292 	 Average reward: -26.159 	 Num_episodes: 22.0
Iteration: 15 	 Objective: -139.690 	 Average reward: -27.179 	 Num_episodes: 23.0
Iteration: 20 	 Objective: -145.278 	 Average reward: -26.804 	 Num_episodes: 21.0
Iteration: 25 	 Objective: -130.501 	 Average reward: -25.007 	 Num_episodes: 24.0
Iteration: 30 	 Objective: -149.344 	 Average reward: -27.901 	 Num_episodes: 21.0
Iteration: 35 	 Objective: -150.483 	 Average reward: -31.208 	 Num_episodes: 20.0
Iteration: 40 	 Objective: -159.771 	 Average reward: -31.942 	 Num_episodes: 20.0
Iteration: 45 	 Objective: -137.978 	 Average reward: -29.539 	 Num_episodes: 22.0
Iteration: 50 	 Objective: -154.424 	 Average reward: -32.620 	 Num_episodes: 20.0
Iteration: 55 	 Objective: -150.200 	 Average reward: -31.036 	 Num_episodes: 2

In [ ]:
if plot:
    fig, axs = plt.subplots(2, 2, figsize=(10,10))
    axs = axs.flatten()
    axs[0].plot(arr_objective)
    axs[0].set_xlabel('Iteration')
    axs[0].set_ylabel('Objective')
    axs[0].set_title(' Objective vs iterations ')
    axs[1].plot(avg_returns)
    axs[1].set_xlabel('Iteration')
    axs[1].set_ylabel('Average Rewards')
    axs[1].set_title(' Average Rewards vs iterations ')
    axs[2].plot(num_episodes)
    axs[2].set_title(' Number of episodes vs iterations ')
    axs[3].plot(sigma[:,0], '-r')
    axs[3].plot(sigma[:,1], '-b')
    axs[3].set_title(' Sigma values vs iterations ')
    axs[3].legend(['sigma1', 'sigma2'])
    
    plt.show()

In [ ]:
if render_env:
    env = gym.make("modified_gym_env:ReacherPyBulletEnv-v1", rand_init=False)
    steps = 0
    env.render('human')
    state = env.reset()
    done = False
    time.sleep(3)
    while steps<300:
        a, _ = pg.getAction(policy_network, state)
        state, r, done, info = env.step(np.array(a))
        steps+=1
        env.render('human')
        time.sleep(0.1)
    env.env.close()

In [ ]:
# policy_network.saveModel(2)